In [9]:
# 安裝所需套件
!pip install pytorch-lightning -q

# 匯入必要的庫
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt
from IPython.display import clear_output

# 設定設備 (使用 GPU 如果可用)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 加載 Iris 數據集
print("Loading the Iris dataset...")
data = load_iris()
X, y = data.data, data.target
print(f"Features shape: {X.shape}, Target shape: {y.shape}")

# 資料標準化
print("Normalizing data...")
scaler = StandardScaler()
X = scaler.fit_transform(X)

# 資料分割 (80% 訓練, 20% 測試)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(f"Training set shape: {X_train.shape}, Testing set shape: {X_test.shape}")

# 將資料轉換為 PyTorch 張量
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
y_test = torch.tensor(y_test, dtype=torch.long)

# 建立 DataLoader
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

# 定義神經網絡模型
class IrisModel(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(4, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(128, 64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(64, 3)
        )
        self.loss_fn = nn.CrossEntropyLoss()

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_pred = self(x)
        loss = self.loss_fn(y_pred, y)
        acc = (y_pred.argmax(dim=1) == y).float().mean()
        self.log('train_loss', loss, prog_bar=True)
        self.log('train_acc', acc, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_pred = self(x)
        loss = self.loss_fn(y_pred, y)
        acc = (y_pred.argmax(dim=1) == y).float().mean()
        self.log('val_loss', loss, prog_bar=True)
        self.log('val_acc', acc, prog_bar=True)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=0.001)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)
        return {'optimizer': optimizer, 'lr_scheduler': scheduler, 'monitor': 'val_loss'}

# 訓練過程中的即時準確率圖表
train_accuracies = []
val_accuracies = []
epochs_list = []

def plot_accuracies():
    clear_output(wait=True)
    plt.figure(figsize=(8, 6))
    plt.plot(epochs_list, train_accuracies, label="Training Accuracy", marker='o')
    plt.plot(epochs_list, val_accuracies, label="Validation Accuracy", marker='x')
    plt.title("Model Accuracy Per Epoch")
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy")
    plt.legend()
    plt.grid(True)
    plt.show()

# 設定訓練器
print("Training the model with PyTorch Lightning...")
early_stopping = EarlyStopping(monitor='val_loss', patience=10, mode='min', verbose=True)
trainer = pl.Trainer(
    max_epochs=100,
    callbacks=[early_stopping],
    log_every_n_steps=1,
    enable_progress_bar=True
)

# 訓練模型
model = IrisModel()
trainer.fit(model, train_dataloaders=train_loader, val_dataloaders=test_loader)

# 評估模型
print("Evaluating the model on the test set...")
results = trainer.validate(model, dataloaders=test_loader)
print(f"Test Accuracy: {results[0]['val_acc']:.4f}")


INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name    | Type             | Params | Mode 
-----------------------------------------------------
0 | model   | Sequential       | 9.5 K  | train
1 | loss_fn | CrossEntropyLoss | 0      | train
-----------------------------------------------------
9.5 K     Trainable params
0         Non-trainable params
9.5 K     Total params
0.038     Total estimated model params size (MB)
11        Modules in train mode
0         Modules in eval mode


Using device: cuda
Loading the Iris dataset...
Features shape: (150, 4), Target shape: (150,)
Normalizing data...
Training set shape: (120, 4), Testing set shape: (30, 4)
Training the model with PyTorch Lightning...


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

INFO:pytorch_lightning.callbacks.early_stopping:Monitored metric val_loss did not improve in the last 10 records. Best score: 0.063. Signaling Trainer to stop.
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Evaluating the model on the test set...


Validation: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃      Validate metric      ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│          val_acc          │            1.0            │
│         val_loss          │    0.07289377599954605    │
└───────────────────────────┴───────────────────────────┘

Test Accuracy: 1.0000
